In [37]:
from PIL import Image
from glob import glob
import pandas as pd
import skimage, os
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
import math
import torch
import cv2
from tqdm.notebook import tqdm, trange


In [38]:
luna_subset_path = 'F:/Nodule_greater_than_5mm/Only_lung_portion/'
file_path_list_all = sorted(glob(luna_subset_path + '*.nii'))

file_path_list_all = file_path_list_all[:280]
print(len(file_path_list_all))


280


In [39]:
df_node = pd.read_csv('E:/Tanvir Mehedi/Luna 16 Dataset/' + "candidates_V2.csv")
df_node


,seriesuid,coordX,coordY,coordZ,class
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,68.420000,-74.480000,-288.700000,0
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-95.209361,-91.809406,-377.426350,0
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-24.766755,-120.379294,-273.361539,0
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-63.080000,-65.740000,-344.240000,0
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,52.946688,-92.688873,-241.067872,0
...,...,...,...,...,...
754970,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,-33.400000,-64.200000,-115.560000,0
754971,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,56.236359,70.352400,-203.446236,0
754972,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,-97.104221,55.738289,-203.879785,0
754973,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,-65.470000,59.670000,-136.370000,0


In [40]:
def load_itk(filename):
    
    # Reads the image using SimpleITK
    itk_image = sitk.ReadImage(filename)

    # Convert the image to a  numpy array first and then shuffle the dimensions to get axis in the order z,y,x
    image_array = np.array(sitk.GetArrayFromImage(itk_image), dtype = np.float32)

    # Read the origin of the ct_scan, will be used to convert the coordinates from world to voxel and vice versa.
    origin = np.array(list(reversed(itk_image.GetOrigin())))

    # Read the spacing along each dimension
    spacing = np.array(list(reversed(itk_image.GetSpacing())))

    return itk_image, image_array, origin, spacing

# For Mask 

def load_itk_mask(filename):
    
    # Reads the image using SimpleITK
    itk_image = sitk.ReadImage(filename)

    # Convert the image to a  numpy array first and then shuffle the dimensions to get axis in the order z,y,x
    image_array = np.array(sitk.GetArrayFromImage(itk_image), dtype = np.uint8)

    # Read the origin of the ct_scan, will be used to convert the coordinates from world to voxel and vice versa.
    origin = np.array(list(reversed(itk_image.GetOrigin())))

    # Read the spacing along each dimension
    spacing = np.array(list(reversed(itk_image.GetSpacing())))

    return itk_image, image_array, origin, spacing


In [41]:
def resample_image(itk_image, out_spacing=[1.0, 1.0, 1.0]):
    
    original_spacing = itk_image.GetSpacing()
    original_size = itk_image.GetSize()

    out_size = [
        int(np.round(original_size[0] * (original_spacing[0] / out_spacing[0]))),
        int(np.round(original_size[1] * (original_spacing[1] / out_spacing[1]))),
        int(np.round(original_size[2] * (original_spacing[2] / out_spacing[2])))
    ]

    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(out_spacing)
    resample.SetSize(out_size)
    resample.SetOutputDirection(itk_image.GetDirection())
    resample.SetOutputOrigin(itk_image.GetOrigin())
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(itk_image.GetPixelIDValue())

    resample.SetInterpolator(sitk.sitkLinear)

    resampled_img = resample.Execute(itk_image)
    resampled_arr = np.array(sitk.GetArrayFromImage(resampled_img), dtype = np.float32)

    return resampled_img,resampled_arr 

# For Mask 

def resample_mask(itk_image, out_spacing=[1.0, 1.0, 1.0]):
    
    original_spacing = itk_image.GetSpacing()
    original_size = itk_image.GetSize()

    out_size = [
        int(np.round(original_size[0] * (original_spacing[0] / out_spacing[0]))),
        int(np.round(original_size[1] * (original_spacing[1] / out_spacing[1]))),
        int(np.round(original_size[2] * (original_spacing[2] / out_spacing[2])))
    ]

    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(out_spacing)
    resample.SetSize(out_size)
    resample.SetOutputDirection(itk_image.GetDirection())
    resample.SetOutputOrigin(itk_image.GetOrigin())
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(itk_image.GetPixelIDValue())

    resample.SetInterpolator(sitk.sitkNearestNeighbor)

    resampled_img = resample.Execute(itk_image)
    resampled_arr = np.array(sitk.GetArrayFromImage(resampled_img), dtype = np.uint8)

    return resampled_img,resampled_arr 


In [42]:
def clipping(resampled_arr):
    
    min_bound = -1200
    max_bound = 600
    
    clipped_arr = np.clip(resampled_arr, min_bound, max_bound)
    
    return clipped_arr


In [43]:
def normalized(clipped_arr):
    
    min_bound = -1200
    max_bound = 600

    normalized_arr = (clipped_arr - min_bound) / (max_bound - min_bound)
    normalized_arr[normalized_arr > 1] = 1.
    normalized_arr[normalized_arr < 0] = 0.
    
    return normalized_arr


In [44]:
def get_cube_from_img(img3d, center_x, center_y, center_z, block_size):
    
    start_x = max(center_x - block_size / 2, 0)
    
    if start_x + block_size > img3d.shape[2]:
        
        start_x = img3d.shape[2] - block_size

    start_y = max(center_y - block_size / 2, 0)
    
    if start_y + block_size > img3d.shape[1]:
        
        start_y = img3d.shape[1] - block_size
        
    start_z = max(center_z - block_size / 2, 0)
    
    if start_z + block_size > img3d.shape[0]:
        
        start_z = img3d.shape[0] - block_size
        
    start_z = int(start_z)
    start_y = int(start_y)
    start_x = int(start_x)
    
    res = img3d[start_z:start_z + block_size, start_y:start_y + block_size, start_x:start_x + block_size]
    
    return res


In [45]:
output_pos_path = 'E:/Tanvir Mehedi/Nodule_Classification/Data/train/pos_cube/'
output_neg_path = 'E:/Tanvir Mehedi/Nodule_Classification/Data/train/neg_cube/'
output_pos_nii_path = 'E:/Tanvir Mehedi/Nodule_Classification/Data/pos_cube_nii/'


In [46]:
block_size = 32


In [47]:
pos_ct = 0
neg_nod = 0
pos_nod = 0

for i in tqdm(range(len(file_path_list_all))):
    
    flag = 0
    
    seriesuid = file_path_list_all[i].split('\\')[-1][:-4]
    itk_image, image_array, origin, spacing = load_itk(file_path_list_all[i])
    
    resampled_img, resampled_arr = resample_image(itk_image, out_spacing=[1.0, 1.0, 1.0])
    clipped_arr = clipping(resampled_arr)
    normalized_arr = normalized(clipped_arr)
    
    direction = np.reshape(itk_image.GetDirection(),[3,3])
    
    mini_df  =  df_node[df_node['seriesuid'] == seriesuid]


    if mini_df.shape[0] > 0:
        
        pos_class = 0
        neg_class = 0
        
        
        for node_idx, cur_row in mini_df.iterrows():

            node_x = cur_row["coordX"]
            node_y = cur_row["coordY"]
            node_z = cur_row["coordZ"]
            class_ = cur_row["class"]
            node_center = np.array([node_x, node_y, node_z])

#             convert x,y,z order v_center to z,y,x order v_center
            node_center[0], node_center[1], node_center[2] = node_center[2], node_center[1], node_center[0]

            node_cent_coord = np.absolute(np.round(((node_center - origin) @ np.linalg.inv(direction)) / [1.0, 1.0, 1.0]))
            
            if class_ == 1:
                
                flag = 1
                pos_class += 1 
                pos_nod += 1
                pos_cube = get_cube_from_img(img3d = normalized_arr, center_x = node_cent_coord[2], 
                                                 center_y = node_cent_coord[1], center_z = node_cent_coord[0], 
                                                 block_size = block_size)
                
                np.save(output_pos_path + seriesuid + '_pos_cube_' + '%04d' %pos_class + '.npy', pos_cube)
                
                img = sitk.GetImageFromArray(pos_cube)

                sitk.WriteImage(img, output_pos_nii_path + seriesuid + '_pos_cube_' + '%04d' %pos_class +".nii")
                
#             else :

            if class_ == 0:

                neg_nod += 1
                neg_class += 1 
                
                neg_cube = get_cube_from_img(img3d = normalized_arr, center_x = node_cent_coord[2], 
                                                 center_y = node_cent_coord[1], center_z = node_cent_coord[0], 
                                                 block_size = block_size)

                np.save(output_neg_path + seriesuid + '_neg_cube_' + '%04d' %neg_class + '.npy', neg_cube)
                
    if flag == 1:

        pos_ct += 1
        

print(i)
print(pos_ct)
    
#     if (neg_nod >= 100000):

#         break


  0%|          | 0/280 [00:00<?, ?it/s]

279
280


In [14]:
output_neg_path = 'E:/Tanvir Mehedi/Nodule_Classification/Data/validation/neg_cube/'

pos_ct = 0
neg_nod = 0

for i in tqdm(range(666,len(file_path_list_all))):
    
    flag = 0
    
    seriesuid = file_path_list_all[i].split('\\')[-1][:-4]
    itk_image, image_array, origin, spacing = load_itk(file_path_list_all[i])
    
    resampled_img, resampled_arr = resample_image(itk_image, out_spacing=[1.0, 1.0, 1.0])
    clipped_arr = clipping(resampled_arr)
    normalized_arr = normalized(clipped_arr)
    
    direction = np.reshape(itk_image.GetDirection(),[3,3])
    
    mini_df  =  df_node[df_node['seriesuid'] == seriesuid]


    if mini_df.shape[0] > 0:
        
        pos_class = 0
        neg_class = 0
        
        
        for node_idx, cur_row in mini_df.iterrows():

            node_x = cur_row["coordX"]
            node_y = cur_row["coordY"]
            node_z = cur_row["coordZ"]
            class_ = cur_row["class"]
            node_center = np.array([node_x, node_y, node_z])

            # convert x,y,z order v_center to z,y,x order v_center
            node_center[0], node_center[1], node_center[2] = node_center[2], node_center[1], node_center[0]

            node_cent_coord = np.absolute(np.round(((node_center - origin) @ np.linalg.inv(direction)) / [1.0, 1.0, 1.0]))


            if class_ == 0:

                neg_nod += 1
                neg_class += 1 
                
                neg_cube = get_cube_from_img(img3d = normalized_arr, center_x = node_cent_coord[2], 
                                                 center_y = node_cent_coord[1], center_z = node_cent_coord[0], 
                                                 block_size = block_size)

                np.save(output_neg_path + seriesuid + '_neg_cube_' + '%04d' %neg_class + '.npy', neg_cube)
                
   
    if (neg_nod >= 2500):

        break




  0%|          | 0/216 [00:00<?, ?it/s]

In [ ]:
print(pos_ct)


In [ ]:
pos_cube_path = sorted(glob(output_pos_path + '*.npy'))

print(len(pos_cube_path))

neg_cube_path = glob(output_neg_path + '*.npy')

print(len(neg_cube_path))


In [ ]:
pos_c = np.load(pos_cube_path[30])

imgs = pos_c

for i in range(len(imgs)):
    
    print("image %d" % i)
    plt.figure(figsize=(5, 5))
    plt.imshow(imgs[i],cmap = 'gray')
    plt.show()
imgs.shape


In [ ]:
neg_c = np.load(neg_cube_path[1000])

imgs = neg_c

for i in range(len(imgs)):
    
    print("image %d" % i)
    plt.figure(figsize=(5,5))
    plt.imshow(imgs[i],cmap='gray')
    plt.show()
imgs.shape


In [ ]:
for i in tqdm(range(len(file_path_list))):
    
    seriesuid = file_path_list[i].split('/')[-1][:-4]
    itk_image, image_array, origin, spacing = load_itk(file_path_list[i])
    
    direction = np.reshape(itk_image.GetDirection(),[3,3])
    
    resampled_img, resampled_arr = resample_image(itk_image, out_spacing=[1.0, 1.0, 1.0])
    
    for j in range(len(seg_file_path_list)):
        
        seg_seriesuid = seg_file_path_list[j].split('/')[-1][:-4]
        
        if seriesuid == seg_seriesuid :
            
            seg_image, seg_array, seg_origin, seg_spacing = load_itk_mask(seg_file_path_list[j])
            resampled_seg_img, resampled_seg_arr = resample_mask(seg_image, out_spacing=[1.0, 1.0, 1.0])
        
            resampled_seg_arr[resampled_seg_arr==3] = 1
            resampled_seg_arr[resampled_seg_arr==4] = 1
            resampled_seg_arr[resampled_seg_arr!=1] = 0
            
            seg_lung = (resampled_arr*resampled_seg_arr).astype(np.float32)

            seg_lung[seg_lung == -0.0] = -1200
            
            clipped_arr = clipping(seg_lung)
            normalized_arr = normalized(clipped_arr)
            direction = np.reshape(itk_image.GetDirection(),[3,3])
    
            mini_df  =  df_node[df_node['seriesuid'] == seriesuid]

            if mini_df.shape[0] == 0:

                print('No')

            if mini_df.shape[0] > 0:

                print('Yes')

                pos_class = 0
                neg_class = 0

                for node_idx, cur_row in mini_df.iterrows():

                    node_x = cur_row["coordX"]
                    node_y = cur_row["coordY"]
                    node_z = cur_row["coordZ"]
                    class_ = cur_row["class"]
                    node_center = np.array([node_x, node_y, node_z])

                    # convert x,y,z order v_center to z,y,x order v_center
                    node_center[0], node_center[1], node_center[2] = node_center[2], node_center[1], node_center[0]

                    node_cent_coord = np.absolute(np.round(((node_center - origin) @ np.linalg.inv(direction)) / [1.0, 1.0, 1.0]))

                    if class_ == 1:

                        pos_class += 1 

                        pos_cube = get_cube_from_img(img3d = normalized_arr, center_x = node_cent_coord[2], 
                                                         center_y = node_cent_coord[1], center_z = node_cent_coord[0], 
                                                         block_size = block_size)

#                         np.save(output_pos_path + seriesuid + '_pos_cube_' + '%02d' %pos_class + '.npy', pos_cube)
                        
                        img = sitk.GetImageFromArray(pos_cube)
    
#                         img.CopyInformation(itk_image)

                        sitk.WriteImage(img, output_pos_nii_path + seriesuid + '_pos_cube_' + '%02d' %pos_class +".nii")

                    else :

                        neg_class += 1 

                        neg_cube = get_cube_from_img(img3d = normalized_arr, center_x = node_cent_coord[2], 
                                                         center_y = node_cent_coord[1], center_z = node_cent_coord[0], 
                                                         block_size = block_size)

#                         np.save(output_neg_path + seriesuid + '_neg_cube_' + '%04d' %neg_class + '.npy', neg_cube)
